<a href="https://colab.research.google.com/github/banunkers/d7041e-labs/blob/master/lab5_ANN_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
#functions of non-linear activations
def f_sigmoid(X, deriv=False):
    if not deriv:
        return 1 / (1 + np.exp(-X))
    else:
        return f_sigmoid(X)*(1 - f_sigmoid(X))


def f_softmax(X):
    Z = np.sum(np.exp(X), axis=1)
    Z = Z.reshape(Z.shape[0], 1)
    return np.exp(X) / Z

def f_relu(x, deriv=False):
    if not deriv:
        #return np.where(x > 0, x, 0)
        return x * (x > 0)
    else:
        #return np.where(x >= 0, 1, 0)
        return 1 * ( x >= 0)

In [0]:
def exit_with_err(err_str):
    print >> sys.stderr, err_str
    sys.exit(1)

In [0]:
#Functionality of a single hidden layer
class Layer:
    def __init__(self, size, batch_size, is_input=False, is_output=False,
                 activation=f_sigmoid):
        self.is_input = is_input
        self.is_output = is_output

        # Z is the matrix that holds output values
        self.Z = np.zeros((batch_size, size[0]))
        # The activation function is an externally defined function (with a
        # derivative) that is stored here
        self.activation = activation

        # W is the outgoing weight matrix for this layer
        self.W = None
        # S is the matrix that holds the inputs to this layer
        self.S = None
        # D is the matrix that holds the deltas for this layer
        self.D = None
        # Fp is the matrix that holds the derivatives of the activation function
        self.Fp = None

        if not is_input:
            self.S = np.zeros((batch_size, size[0]))
            self.D = np.zeros((batch_size, size[0]))

        if not is_output:
            self.W = np.random.normal(size=size, scale=1E-4)

        if not is_input and not is_output:
            self.Fp = np.zeros((size[0], batch_size))

    def forward_propagate(self):
        if self.is_input:
            return self.Z.dot(self.W)

        self.Z = self.activation(self.S)
        if self.is_output:
            return self.Z
        else:
            # For hidden layers, we add the bias values here
            self.Z = np.append(self.Z, np.ones((self.Z.shape[0], 1)), axis=1)
            self.Fp = self.activation(self.S, deriv=True).T
            return self.Z.dot(self.W)


In [0]:
class MultiLayerPerceptron:
    def __init__(self, layer_config, batch_size=100, activation=f_sigmoid):
        self.layers = []
        self.num_layers = len(layer_config)
        self.minibatch_size = batch_size

        for i in range(self.num_layers-1):
            if i == 0:
                print ("Initializing input layer with size {0}.".format(layer_config[i]))
                # Here, we add an additional unit at the input for the bias
                # weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         is_input=True))
            else:
                print ("Initializing hidden layer with size {0}.".format(layer_config[i]))
                # Here we add an additional unit in the hidden layers for the
                # bias weight.
                self.layers.append(Layer([layer_config[i]+1, layer_config[i+1]],
                                         batch_size,
                                         activation=activation))

        print ("Initializing output layer with size {0}.".format(layer_config[-1]))
        self.layers.append(Layer([layer_config[-1], None],
                                 batch_size,
                                 is_output=True,
                                 activation=f_softmax))
        print ("Done!")

    def forward_propagate(self, data):
        # We need to be sure to add bias values to the input
        self.layers[0].Z = np.append(data, np.ones((data.shape[0], 1)), axis=1)

        for i in range(self.num_layers-1):
            self.layers[i+1].S = self.layers[i].forward_propagate()
        return self.layers[-1].forward_propagate()

    def backpropagate(self, yhat, labels):
        
        #exit_with_err("FIND ME IN THE CODE, What is computed in the next line of code?\n")
        # Calculates the error of the output (loss function)
        
        self.layers[-1].D = (yhat - labels).T

        for i in range(self.num_layers-2, 0, -1):
            # We do not calculate deltas for the bias values
            W_nobias = self.layers[i].W[0:-1, :]
            
            #exit_with_err("FIND ME IN THE CODE, What does this 'for' loop do?\n") 
            # Calculates the error of every node in the layer
            
            self.layers[i].D = W_nobias.dot(self.layers[i+1].D) * self.layers[i].Fp

    def update_weights(self, eta):
        for i in range(0, self.num_layers-1):
            W_grad = -eta*(self.layers[i+1].D.dot(self.layers[i].Z)).T
            self.layers[i].W += W_grad

    def evaluate(self, train_data, train_labels, test_data, test_labels,
                 num_epochs=70, eta=0.05, eval_train=False, eval_test=True):

        N_train = len(train_labels)*len(train_labels[0])
        N_test = len(test_labels)*len(test_labels[0])

        print ("Training for {0} epochs...".format(num_epochs))
        for t in range(0, num_epochs):
            out_str = "[{0:4d}] ".format(t)

            for b_data, b_labels in zip(train_data, train_labels):
                output = self.forward_propagate(b_data)
                self.backpropagate(output, b_labels)
                
                #exit_with_err("FIND ME IN THE CODE, How does weight update is implemented? What is eta?\n") 
                # update weights performs gradient decent. ETA is the learning rate

                self.update_weights(eta=eta)

            if eval_train:
                errs = 0
                for b_data, b_labels in zip(train_data, train_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Training error: {1:.5f}".format(out_str,
                                                           float(errs)/N_train))

            if eval_test:
                errs = 0
                for b_data, b_labels in zip(test_data, test_labels):
                    output = self.forward_propagate(b_data)
                    yhat = np.argmax(output, axis=1)
                    errs += np.sum(1-b_labels[np.arange(len(b_labels)), yhat])

                out_str = ("{0} Test error: {1:.5f}").format(out_str,
                                                       float(errs)/N_test)

            print (out_str)


In [0]:
def label_to_bit_vector(labels, nbits):
    bit_vector = np.zeros((labels.shape[0], nbits))
    for i in range(labels.shape[0]):
        bit_vector[i, labels[i]] = 1.0

    return bit_vector

In [0]:
def create_batches(data, labels, batch_size, create_bit_vector=False):
    N = data.shape[0]
    print ("Batch size {0}, the number of examples {1}.".format(batch_size,N))

    if N % batch_size != 0:
        print ("Warning in create_minibatches(): Batch size {0} does not " \
              "evenly divide the number of examples {1}.".format(batch_size,N))
    chunked_data = []
    chunked_labels = []
    idx = 0
    while idx + batch_size <= N:
        chunked_data.append(data[idx:idx+batch_size, :])
        if not create_bit_vector:
            chunked_labels.append(labels[idx:idx+batch_size])
        else:
            bit_vector = label_to_bit_vector(labels[idx:idx+batch_size], 10)
            chunked_labels.append(bit_vector)

        idx += batch_size

    return chunked_data, chunked_labels


In [0]:
def prepare_for_backprop(batch_size, Train_images, Train_labels, Valid_images, Valid_labels):
    
    print ("Creating data...")
    batched_train_data, batched_train_labels = create_batches(Train_images, Train_labels,
                                              batch_size,
                                              create_bit_vector=True)
    batched_valid_data, batched_valid_labels = create_batches(Valid_images, Valid_labels,
                                              batch_size,
                                              create_bit_vector=True)
    print ("Done!")


    return batched_train_data, batched_train_labels,  batched_valid_data, batched_valid_labels



In [0]:
from keras.datasets import mnist

In [33]:
(Xtr, Ltr), (X_test, L_test)=mnist.load_data()

Xtr = Xtr.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
Xtr = Xtr.astype('float32')
X_test = X_test.astype('float32')
Xtr /= 255
X_test /= 255
print(Xtr.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


60000 train samples
10000 test samples


In [37]:
# default config
batch_size=100;

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)

mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)

mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eval_train=True)

print("Done:)\n")


Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.38237 Test error: 0.37960
[   1]  Training error: 0.07925 Test error: 0.07540
[   2]  Training error: 0.04930 Test error: 0.05520
[   3]  Training error: 0.04367 Test error: 0.05050
[   4]  Training error: 0.03482 Test error: 0.04360
[   5]  Training error: 0.03140 Test error: 0.03960
[   6]  Training error: 0.02665 Test error: 0.03640
[   7]  Training error: 0.02392 Test error: 0.03650
[   8]  Training error: 0.02075 Test error: 0.03520
[   9]  Training error: 0.01900 Test error: 0.03270
[  10]  Training error: 0.01757 Test error: 0.03210
[  11]  Training error: 0.02128 Test error: 0.03690
[  12]  Training error: 0.01447 Test error: 0.03170
[  13]  Trainin

In [39]:
eta = 0.005
print("Learning rate = ", eta)

train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)
mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)
mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eval_train=True, eta=eta)

Learning rate =  0.005
Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.70333 Test error: 0.70060
[   1]  Training error: 0.64720 Test error: 0.64300
[   2]  Training error: 0.59970 Test error: 0.59910
[   3]  Training error: 0.45720 Test error: 0.46770
[   4]  Training error: 0.21008 Test error: 0.20050
[   5]  Training error: 0.11277 Test error: 0.11060
[   6]  Training error: 0.08818 Test error: 0.08870
[   7]  Training error: 0.07360 Test error: 0.07470
[   8]  Training error: 0.06233 Test error: 0.06390
[   9]  Training error: 0.05303 Test error: 0.05440
[  10]  Training error: 0.04668 Test error: 0.04760
[  11]  Training error: 0.04140 Test error: 0.04280
[  12]  Training error: 0.03670 Test error: 

In [40]:
eta = 0.5
print("Learning rate = ", eta)
train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)
mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size)
mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eval_train=True, eta=eta)

print("Done:)\n")

Learning rate =  0.5
Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.90965 Test error: 0.91080
[   1]  Training error: 0.90128 Test error: 0.90200
[   2]  Training error: 0.89558 Test error: 0.89720
[   3]  Training error: 0.90070 Test error: 0.89680
[   4]  Training error: 0.90248 Test error: 0.90260
[   5]  Training error: 0.90085 Test error: 0.89910
[   6]  Training error: 0.89782 Test error: 0.89900
[   7]  Training error: 0.90085 Test error: 0.89910
[   8]  Training error: 0.90248 Test error: 0.90260
[   9]  Training error: 0.89782 Test error: 0.89900
[  10]  Training error: 0.88763 Test error: 0.88650
[  11]  Training error: 0.90965 Test error: 0.91080
[  12]  Training error: 0.89782 Test error: 0.

# Difference learning rate 0.5 and 0.005
The learning rate of 0.5 is too high and causes the model to diconverge

In [47]:
print("ReLU output function")
train_data, train_labels, valid_data, valid_labels=prepare_for_backprop(batch_size, Xtr, Ltr, X_test, L_test)
mlp = MultiLayerPerceptron(layer_config=[784, 100, 100, 10], batch_size=batch_size, activation=f_relu)
mlp.evaluate(train_data, train_labels, valid_data, valid_labels, eval_train=True, eta=eta)

print("Done:)\n")

ReLU output function
Creating data...
Batch size 100, the number of examples 60000.
Batch size 100, the number of examples 10000.
Done!
Initializing input layer with size 784.
Initializing hidden layer with size 100.
Initializing hidden layer with size 100.
Initializing output layer with size 10.
Done!
Training for 70 epochs...
[   0]  Training error: 0.89558 Test error: 0.89720
[   1]  Training error: 0.89782 Test error: 0.89900
[   2]  Training error: 0.90128 Test error: 0.90200
[   3]  Training error: 0.89782 Test error: 0.89900
[   4]  Training error: 0.90248 Test error: 0.90260
[   5]  Training error: 0.90128 Test error: 0.90200
[   6]  Training error: 0.89782 Test error: 0.89900
[   7]  Training error: 0.90128 Test error: 0.90200
[   8]  Training error: 0.89782 Test error: 0.89900
[   9]  Training error: 0.90965 Test error: 0.91080
[  10]  Training error: 0.90085 Test error: 0.89910
[  11]  Training error: 0.90128 Test error: 0.90200
[  12]  Training error: 0.88763 Test error: 0.